In [17]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

In [18]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import itertools
from tqdm import tqdm

In [19]:
df = pd.read_csv(r"C:\Users\mikha\Dropbox\mikhael_misc\Projects\Policing Thesis\Modified Dataset.csv")

In [20]:
race_set = {'ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'WHITE'}
race_set_w_total = {'ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'TOTAL', 'WHITE'}

# *coordinates_dict*

*coordinates_dict[ longitude_i ][ latitude_j ][ FIELD** ][ race ] = information for that area*

** "field" equals things like "Number of Charges @ 9 MPH", "Average Speed NOT in 9,14 MPH", etc

## Set up *coordinates_dict*

In [21]:
CELL_WIDTH = 0.1
DECIMAL_PLACES = 2

MIN_LONGITUDE = -77.6
MAX_LONGITUDE = -76.8

MIN_LATITUDE = 38.92
MAX_LATITUDE = 39.5

def rounder(x):#rounds to nearest CELL_WIDTH; X decimal places
    return np.around(CELL_WIDTH * round(x/CELL_WIDTH), DECIMAL_PLACES)

TOTAL_CELLS = round((((MIN_LONGITUDE-CELL_WIDTH)-(MAX_LONGITUDE+CELL_WIDTH))/CELL_WIDTH)*(((MIN_LATITUDE-CELL_WIDTH)-(MAX_LATITUDE+CELL_WIDTH))/CELL_WIDTH))
print('total # cells ~ ', TOTAL_CELLS)

coordinates_dict = {}
for x in np.arange(MIN_LONGITUDE-CELL_WIDTH, MAX_LONGITUDE+CELL_WIDTH, CELL_WIDTH):
    coordinates_dict[rounder(np.around(x,DECIMAL_PLACES))] = {}

for x in coordinates_dict:
    for y in np.arange(MIN_LATITUDE-CELL_WIDTH, MAX_LATITUDE+CELL_WIDTH, CELL_WIDTH):
        coordinates_dict[x][rounder(np.around(y,DECIMAL_PLACES))] = {"All Speeds Over Limit": {race: [] for race in race_set_w_total},
                                               "Number of Speeding Charges":{race: 0 for race in race_set_w_total},
                                               "Average Speed NOT in 9,14 MPH": {race: 0 for race in race_set_w_total},
                                               
                                                 "Number of Charges @ 9 MPH": {race: 0 for race in race_set_w_total},
                                                "Number of Charges in 9,14 MPH": {race: 0 for race in race_set_w_total},
                                                 "Number of Charges in 10,14 MPH": {race: 0 for race in race_set_w_total},
                                                "Number of Charges NOT in 9,14 MPH": {race: 0 for race in race_set_w_total}, 
                                                
                                                 "Percent of Own Race's Charges @ 9 MPH": {race: 0 for race in race_set_w_total},
                                                "Percent of Own Race's Charges in 9,14 MPH": {race: 0 for race in race_set_w_total},
                                                 "Percent of Own Race's Charges in 10,14 MPH": {race: 0 for race in race_set_w_total},
                                                "Percent of Own Race's Charges NOT in 9,14 MPH": {race: 0 for race in race_set_w_total}, 

                                                "Percent of Charges that were CHANGED": {race: 0 for race in race_set_w_total}, 
                                                "Percent of Charges that were NOT CHANGED": {race: 0 for race in race_set_w_total}, 
                                                                    
#                                                  "As Percent of All Races' Charges @ 9 MPH" : {race: 0 for race in race_set_w_total},
#                                                 "As Percent of All Races' in 9,14 MPH": {race: 0 for race in race_set_w_total},
#                                                  "As Percent of All Races in 10,14 MPH": {race: 0 for race in race_set_w_total},
#                                                 "As Percent of All Races in 9,14 MPH": {race: 0 for race in race_set_w_total},                                                                      
                                                
                                                "Number of Citations for Each Stop": {race: [] for race in race_set_w_total},
                                                "Racial Composition": {race: 0 for race in race_set_w_total}}

        
        


total # cells ~  78


### Def Rounding Function

In [22]:
def rounder(x):
    """
    Rounds to nearest CELL_WIDTH; decimal places
    """
    
    return np.around(CELL_WIDTH * round(x/CELL_WIDTH), DECIMAL_PLACES)

## Fill *coordinates_dict*

In [23]:
# zipped_vars=list(zip(df['Longitude'], df['Latitude'], df['Race'], df['Speed Over Posted Limit'], df['Number of Citations']))

for i,x in enumerate(list(zip(df['Longitude'], df['Latitude'], df['Race'], df['Speed Over Posted Limit'], df['Number of Citations']))):
        if MIN_LONGITUDE <= x[0] <= MAX_LONGITUDE: #-77.6 <= x <= -76.8
            if MIN_LATITUDE <= x[1] <= MAX_LATITUDE:

                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Racial Composition" ][ x[2] ] += 1
                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Racial Composition" ][ 'TOTAL' ] += 1

                
                if not np.isnan(x[3]): # if df["Speed Over Posted Limit"].iloc[i] != np.nan
                    
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ x[2] ] += 1
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ 'TOTAL' ] += 1

                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "All Speeds Over Limit" ][ x[2] ].append(x[3])
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "All Speeds Over Limit" ]['TOTAL'].append(x[3])

                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ x[2] ] += 1
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ 'TOTAL' ] +=1

                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Citations for Each Stop" ][ x[2] ].append(x[4])
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Citations for Each Stop" ][ 'TOTAL' ].append(x[4])


                    if x[3] == 9:
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges @ 9 MPH" ][ x[2] ] += 1
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges @ 9 MPH" ][ 'TOTAL' ] += 1

                    if 9 <= x[3] <= 14:
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 9,14 MPH" ][ x[2] ] += 1
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 9,14 MPH" ][ 'TOTAL' ] += 1

                    if 10 <= x[3] <= 14:
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 10,14 MPH" ][ x[2] ] += 1
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 10,14 MPH" ][ 'TOTAL' ] += 1
                        
                    if 9 > x[3] or x[3] > 14:
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges NOT in 9,14 MPH" ][ x[2] ] += 1
                        coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges NOT in 9,14 MPH" ][ 'TOTAL' ] += 1

## Further Modifications to Data

In [24]:
"""
DO NOT THROW AWAY - THIS IS USED
"""

def divide(a,b):
    if a!=0 and b!=0:
        return (a/b)
    else:
        return 0


for long in coordinates_dict:
    for lat in coordinates_dict[long]:
        for race in race_set_w_total:
            coordinates_dict[long][lat]['Average Speed NOT in 9,14 MPH'][race] = np.mean([x for x in coordinates_dict[long][lat]['All Speeds Over Limit'][race] if x<9 or x>14])
            coordinates_dict[long][lat]["Number of Citations for Each Stop"][race] = np.mean(coordinates_dict[long][lat]["Number of Citations for Each Stop"][race])
            
            coordinates_dict[long][lat]["Percent of Own Race's Charges @ 9 MPH"][race] = divide(coordinates_dict[long][lat]["Number of Charges @ 9 MPH"][race], coordinates_dict[long][lat]["Number of Speeding Charges"][race])
            coordinates_dict[long][lat]["Percent of Own Race's Charges in 9,14 MPH"][race] = divide(coordinates_dict[long][lat]["Number of Charges in 9,14 MPH"][race], coordinates_dict[long][lat]["Number of Speeding Charges"][race])
            coordinates_dict[long][lat]["Percent of Own Race's Charges in 10,14 MPH"][race] = divide(coordinates_dict[long][lat]["Number of Charges in 10,14 MPH"][race], coordinates_dict[long][lat]["Number of Speeding Charges"][race])
            coordinates_dict[long][lat]["Percent of Own Race's Charges NOT in 9,14 MPH"][race] = divide(coordinates_dict[long][lat]["Number of Charges NOT in 9,14 MPH"][race], coordinates_dict[long][lat]["Number of Speeding Charges"][race])
            
            coordinates_dict[long][lat]["Percent of Charges that were CHANGED"][race] = divide(coordinates_dict[long][lat]["Number of Charges @ 9 MPH"][race], coordinates_dict[long][lat]["Number of Charges in 9,14 MPH"][race])
            coordinates_dict[long][lat]["Percent of Charges that were NOT CHANGED"][race] = divide(coordinates_dict[long][lat]["Number of Charges in 10,14 MPH"][race], coordinates_dict[long][lat]["Number of Charges in 9,14 MPH"][race])
                        
        for race in race_set:
            coordinates_dict[long][lat]["Racial Composition"][race] = divide(coordinates_dict[long][lat]["Racial Composition"][race], coordinates_dict[long][lat]["Racial Composition"]['TOTAL'])
                        

            
#             "Percent of Own Race's Charges @ 9 MPH": {race: 0 for race in race_set_w_total},
#             "Percent of Own Race's Charges in 9,14 MPH": {race: 0 for race in race_set_w_total},
#             "Percent of Own Race's Charges in 10,14 MPH": {race: 0 for race in race_set_w_total},
#             "Percent of Own Race's Charges NOT in 9,14 MPH": {race: 0 for race in race_set_w_total}, 

#             "As Percent of All Races' Charges @ 9 MPH" : {race: 0 for race in race_set_w_total},
#             "As Percent of All Races' in 9,14 MPH": {race: 0 for race in race_set_w_total},
#             "As Percent of All Races in 10,14 MPH": {race: 0 for race in race_set_w_total},
#             "As Percent of All Races in 9,14 MPH": {race: 0 for race in race_set_w_total},            
                
#             "PERCENT OF CHARGES THAT COULD HAVE BEEN CHANGED"

        

# Creating *coordinates_df* out of *coordinates_dict*

## Create Longitude, Latitude vectors for *coordinates_df*

In [25]:
long_list = [rounder(x) for x in coordinates_dict.keys()]
lat_list = [rounder(x) for x in coordinates_dict[rounder(MIN_LONGITUDE)].keys()]

long_list_for_df = [[x]*len(lat_list) for x in long_list] # all longitudes repeated N times; N=Number of Latitudinal Entries
lat_list_for_df = [lat_list for x in long_list]

#flatten lists
long_list_for_df = np.array(list(itertools.chain.from_iterable(long_list_for_df)))
lat_list_for_df = np.array(list(itertools.chain.from_iterable(lat_list_for_df)))

## Assign Values to rows of *coordinates_df*

In [27]:
coordinates_df = pd.DataFrame({'Longitude':long_list_for_df, 'Latitude':lat_list_for_df})

#STILL PUTTING LISTS IN AS ELEMENTS
scalar_vars_list = [
"Number of Speeding Charges",
"Average Speed NOT in 9,14 MPH",
"Number of Citations for Each Stop",
"Racial Composition",
"Percent of Charges that were CHANGED",
"Percent of Charges that were NOT CHANGED",

#LISTS BELOW
"All Speeds Over Limit"    
]

def nanner(a):
    """
    Makes sure np.nan and empty lists don't pass through
    """
    
    if type(a) != list:
        if np.isnan(a):
            return 0
        else:
            return a
    else: 
        return a

# for i, longlat in enumerate(zip(coordinates_df['Longitude'], coordinates_df['Latitude'])):
for race in race_set_w_total:
    for var in scalar_vars_list:
        coordinates_df[race + ' ' + var] = [nanner(coordinates_dict[longlat[0]][longlat[1]][var][race]) for longlat in zip(coordinates_df['Longitude'], coordinates_df['Latitude'])]
    
    


## Replace 0-Values with *nan*

**NOT DOING THIS ANYMORE - REGRESSION DOESN'T ACCEPT NP.NAN**

In [ ]:
# coordinates_df.replace(0, np.nan, inplace=True)

# Link Geo Data to Main DF 

## Create *concat_df* which will be concatenated with original *df*

In [28]:
concat_df = pd.DataFrame({'Longitude':df['Longitude'], 'Latitude':df['Latitude']})

#STILL PUTTING LISTS IN AS ELEMENTS
scalar_vars_list = [
"Number of Speeding Charges",
"Average Speed NOT in 9,14 MPH",
"Number of Citations for Each Stop",
"Racial Composition",
"Percent of Charges that were CHANGED",
"Percent of Charges that were NOT CHANGED",

#LISTS BELOW
# "All Speeds Over Limit"    
]

def strong_lat_rounder(x):
    """
    Return rounded latitude IF latitude is inside of Montgomery County; else return np.nan
    """
    if MIN_LATITUDE <= rounder(x) <= MAX_LATITUDE:
        return rounder(x)
    else:
        return np.nan
    
def strong_lon_rounder(x):
    if MIN_LONGITUDE <= rounder(x) <= MAX_LONGITUDE:
        return rounder(x)
    else:
        return np.nan

from tqdm import tqdm
for race in tqdm(race_set_w_total):
    for var in scalar_vars_list:
        temp_col = []
        for longlatrace in zip(concat_df['Longitude'], concat_df['Latitude'], df['Race'], df['Speed Over Posted Limit']):
            if not np.isnan(longlatrace[3]): #if this is a speeding ticket
                try:
                    temp_col.append(coordinates_dict[strong_lon_rounder(longlatrace[0])][strong_lat_rounder(longlatrace[1])][var][longlatrace[2]])
                except:
                    temp_col.append(np.nan)
            else:
                temp_col.append(np.nan)
        concat_df['GEO: ' + race + ' ' + var] = temp_col
        

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [04:26<00:00, 38.12s/it]


## Merge DataFrames

In [29]:
df = pd.concat([df, concat_df], axis=1, sort=False)

df = df[df['Speed Over Posted Limit'] > 0]

# del duplicated Long, Lat columns
df = df.loc[:,~df.columns.duplicated()]

In [ ]:
df.head()

In [ ]:
coordinates_df.corr()

# Export

Maybe consider naming cells after numberof *non-empty* cells

In [30]:
FILENAME_STR = f"Geo Modified Dataset - {str(len(coordinates_df))} cells - width=={CELL_WIDTH} .csv"

df.to_csv(FILENAME_STR, index=True)

In [31]:
FILENAME_STR_2 = f"Purely Geographical - Speeding Dataset - {str(len(coordinates_df))} cells - width=={CELL_WIDTH} .csv"

coordinates_df.to_csv(FILENAME_STR_2)

# Prelim Analysis (not done)

### Sample Size per Cell

In [ ]:
coordinates_df['TOTAL Number of Speeding Charges'].hist(log=True, bins=100)

In [ ]:
coordinates_df['TOTAL Number of Speeding Charges'].plot.hist(ylim=(0,100), bins=100) 

### Number of Empty Cells

In [ ]:
print('Percent of Empty Cells == ', sum([1 for x in coordinates_df['TOTAL Number of Speeding Charges'] if x==0])/len(coordinates_df))
print("Number of Non-Empty Cells == ", sum([1 for x in coordinates_df['TOTAL Number of Speeding Charges'] if x!=0]))

In [ ]:
coordinates_df['TOTAL Number of Speeding Charges'].hist(bins=100) 

### Weighted Correlation

In [ ]:
def weighted_mean(x, w):
    return np.sum(x * w) / np.sum(w)

def weighted_cov(x, y, w):
    return np.sum(w * (x - weighted_mean(x, w)) * (y - weighted_mean(y, w))) / np.sum(w)

def weighted_corr(x, y, w):
    return weighted_cov(x, y, w) / np.sqrt(weighted_cov(x, x, w) * weighted_cov(y, y, w))

In [ ]:
print('normal corr == ', np.corrcoef(coordinates_df['WHITE Racial Composition'],coordinates_df['BLACK Percent of Charges that were NOT CHANGED']))
print('weighted_corr == ', weighted_corr(coordinates_df['WHITE Racial Composition'],coordinates_df['BLACK Percent of Charges that were NOT CHANGED'], coordinates_df['TOTAL Number of Speeding Charges']))

In [ ]:
#cross-corr table

coordinates_df.corr()